### 과제 2: 배송 성과 분석
**과제 2: 배송 성과 분석 및 지역별 물류 최적화 방안**
주문부터 배송 완료까지의 시간을 분석하고, 지역별 배송 성과 차이를 파악하여 물류 최적화 방안을 도출하세요. 배송 지연이 고객 만족도(리뷰 점수)에 미치는 영향도 함께 분석하세요



## EDA 진행 
**목표:** 지역별 배송 시간의 평균과 분포를 확인하고, 배송 시간과 고객 리뷰 점수 간의 관계를 파악합니다.

### **분석 로직:**

1. **날짜/시간 변환**: `orders` 데이터셋의 `order_purchase_timestamp`와 `order_delivered_customer_date`를 `datetime` 타입으로 변환합니다.
2. **배송 시간 계산**: `order_delivered_customer_date`에서 `order_purchase_timestamp`를 빼서 배송 소요 시간을 계산합니다.
3. **데이터 병합**: `orders`와 `geolocation` 데이터셋을 병합하여 주문별 지역 정보를 추가합니다. 또한, `orders`와 `reviews` 데이터셋을 병합하여 주문별 리뷰 점수를 연결합니다.
4. **지역별 분석**: `customer_state`를 기준으로 그룹화하여 지역별 평균 배송 시간을 계산합니다.
5. **상관관계 분석**: 배송 소요 시간과 `review_score` 간의 상관계수를 계산합니다.

### **시각화 계획:**

- **박스플롯**: 지역별(주/도시) 배송 시간의 분포를 한눈에 비교하기 위해 박스플롯을 그립니다.
- **산점도**: 배송 소요 시간과 리뷰 점수 간의 관계를 시각적으로 확인하기 위해 산점도를 그립니다.
- **히스토그램**: 리뷰 점수별(1~5점) 배송 소요 시간의 분포를 비교하는 히스토그램을 그립니다.


오늘 진행할 내용은  
**과제 2: 배송 성과 분석 및 지역별 물류 최적화 방안**
주문부터 배송 완료까지의 시간을 분석하고, 지역별 배송 성과 차이를 파악하여 물류 최적화 방안을 도출하세요. 배송 지연이 고객 만족도(리뷰 점수)에 미치는 영향도 함께 분석 




# 데이터 전처리 코드 

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
# 폰트 깨짐 방지: Matplotlib에 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)
# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 주요 테이블 로딩

customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
category_name = pd.read_csv("product_category_name_translation.csv")
geolocation= pd.read_csv("olist_geolocation_dataset.csv")


### 데이터 형 변환 (포르투갈어-> 영어, date_time 형식으로 변환)

In [ ]:
# 각 데이터셋 크기 확인
datasets = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "products": products,
    "sellers": sellers,
    "payments": payments,
    "reviews": reviews,
    "category_name": category_name,
    "geolocation": geolocation
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")
    display(df.head(2))  # 앞부분 2행만 미리보기


# ======================
# [1] Products 카테고리명 번역 (포르투갈어 → 영어)
# ======================
products = pd.read_csv("olist_products_dataset.csv")
category_translation = pd.read_csv("product_category_name_translation.csv")

# 병합 후 컬럼 정리
products = (
    products.merge(category_translation, on="product_category_name", how="left")
            .drop("product_category_name", axis=1)
            .rename(columns={"product_category_name_english": "product_category_name"})
)

print("\n=== 변환 후 'products' 카테고리 예시 ===")
print(products["product_category_name"].head())


# ======================
# [2] Orders 날짜 컬럼 변환
# ======================
datetime_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
# orders 데이터셋의 모든 날짜/시간 관련 컬럼을 datetime 타입으로 변환
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])



# ======================
# [3] Reviews 날짜 컬럼 변환
# ======================
reviews[["review_creation_date", "review_answer_timestamp"]] = reviews[
    ["review_creation_date", "review_answer_timestamp"]
].apply(pd.to_datetime, errors="coerce")

print("\n=== Reviews 날짜 컬럼 dtype 확인 ===")
print(reviews[["review_creation_date", "review_answer_timestamp"]].dtypes)




### 결측치 처리

- orders: 배송 완료일(order_delivered_customer_date) 없는 주문 제거 ✅
- orders: 결제 승인일(order_approved_at) 없는 주문 제거 ✅
- reviews: 코멘트 결측치 → "no comment" 대체 ✅

In [ ]:
# orders 데이터셋을 불러옵니다.
# 사용자 정의 변수명을 참조하여 코드를 작성합니다.
print("=== 'orders' 데이터셋 결측치 현황 ===")
print(orders.isnull().sum())

# 배송 완료일(`order_delivered_customer_date`)이 없는 주문은 배송이 완료되지 않은 상태입니다.
# 배송 성과 분석을 위해 이 행들을 제거합니다.
orders.dropna(subset=['order_delivered_customer_date'], inplace=True)
print("\n배송 미완료 주문 제거 후 데이터 크기:", orders.shape)

# 추가적으로, 결제 승인일(`order_approved_at`)이 없는 주문도 분석에서 제외할 수 있습니다.
# 이는 유효하지 않은 주문으로 간주될 수 있기 때문입니다.
orders.dropna(subset=['order_approved_at'], inplace=True)
print("\n결제 미승인 주문 제거 후 최종 데이터 크기:", orders.shape)
print("\n결제 미승인 주문 제거 후 최종 데이터 크기:", orders .shape)


# 결측치가 제거된 최종 데이터셋의 결측치 현황을 다시 확인합니다.
print("\n=== 최종 데이터셋 결측치 재확인 ===")
print(orders.isnull().sum())

# 결측치가 제거된 데이터셋의 상위 5개 행을 확인합니다.
print("\n=== 최종 데이터셋 미리보기 ===")
print(orders.head())

# reviews 데이터셋을 불러옵니다.
reviews = pd.read_csv('olist_order_reviews_dataset.csv')

print("=== 'olist_order_reviews_dataset.csv' 결측치 현황 ===")
print(reviews.isnull().sum())

# 코멘트 관련 컬럼(`review_comment_title`, `review_comment_message`)의 결측치는
# 고객이 코멘트를 남기지 않았음을 의미하므로, 'no comment'로 채울 수 있습니다.
reviews['review_comment_title'].fillna('no comment', inplace=True)
reviews['review_comment_message'].fillna('no comment', inplace=True)

print("\n결측치 처리 후 현황:")
print(reviews.isnull().sum())

## 이상값 처리

- 이상치 처리
    - orders: 배송 소요일(delivery_time_days) IQR 기반 제거 ✅
    - order_items: 가격(price) IQR 기반 제거 ✅
    - products: 무게(product_weight_g) IQR 기반 제거 ✅
    - payments: 결제 금액(payment_value) IQR 기반 제거 ✅

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터셋 불러오기
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")

# 폰트 깨짐 방지: Matplotlib에 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# orders 데이터 날짜 변환 및 배송 소요 시간 계산
orders['order_purchase_timestamp'] = pd.to_datetime#(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders.dropna(subset=['order_delivered_customer_date'], inplace=True)
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 시각화를 위한 원본 데이터 복사
orders_original = orders.copy()
order_items_original = order_items.copy()
products_original = products.copy()
payments_original = payments.copy()

# =================================================================
# 이상값 처리 및 시각화
# =================================================================

fig, axes = plt.subplots(2, 4, figsize=(15, 10))
fig.suptitle('이상값 처리 전후 박스플롯 비교', fontsize=16)

# ----------------- 배송 소요 시간 (orders) -----------------
# 처리 전
sns.boxplot(x=orders_original['delivery_time_days'], ax=axes[0, 0])
axes[0, 0].set_title('배송 소요 시간 (처리 전)')
axes[0, 0].set_xlabel('일수')

# 이상값 처리
Q1 = orders['delivery_time_days'].quantile(0.25)
Q3 = orders['delivery_time_days'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
orders_cleaned = orders[(orders['delivery_time_days'] >= lower_bound) & (orders['delivery_time_days'] <= upper_bound)]

# 처리 후
sns.boxplot(x=orders_cleaned['delivery_time_days'], ax=axes[1, 0])
axes[1, 0].set_title('배송 소요 시간 (처리 후)')
axes[1, 0].set_xlabel('일수')

# ----------------- 상품 가격 (order_items) -----------------
# 처리 전
sns.boxplot(x=order_items_original['price'], ax=axes[0, 1])
axes[0, 1].set_title('상품 가격 (처리 전)')
axes[0, 1].set_xlabel('가격 (R$)')

# 이상값 처리
Q1 = order_items['price'].quantile(0.25)
Q3 = order_items['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
order_items_cleaned = order_items[(order_items['price'] >= lower_bound) & (order_items['price'] <= upper_bound)]

# 처리 후
sns.boxplot(x=order_items_cleaned['price'], ax=axes[1, 1])
axes[1, 1].set_title('상품 가격 (처리 후)')
axes[1, 1].set_xlabel('가격 (R$)')

# ----------------- 상품 무게 (products) -----------------
# 처리 전
sns.boxplot(x=products_original['product_weight_g'], ax=axes[0, 2])
axes[0, 2].set_title('상품 무게 (처리 전)')
axes[0, 2].set_xlabel('무게 (g)')

# 이상값 처리
Q1 = products['product_weight_g'].quantile(0.25)
Q3 = products['product_weight_g'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
products_cleaned = products[(products['product_weight_g'] >= lower_bound) & (products['product_weight_g'] <= upper_bound)]

# 처리 후
sns.boxplot(x=products_cleaned['product_weight_g'], ax=axes[1, 2])
axes[1, 2].set_title('상품 무게 (처리 후)')
axes[1, 2].set_xlabel('무게 (g)')

# ----------------- 결제 금액 (payments) -----------------
# 처리 전
sns.boxplot(x=payments_original['payment_value'], ax=axes[0, 3])
axes[0, 3].set_title('결제 금액 (처리 전)')
axes[0, 3].set_xlabel('금액 (R$)')

# 이상값 처리
Q1 = payments['payment_value'].quantile(0.25)
Q3 = payments['payment_value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
payments_cleaned = payments[(payments['payment_value'] >= lower_bound) & (payments['payment_value'] <= upper_bound)]

# 처리 후
sns.boxplot(x=payments_cleaned['payment_value'], ax=axes[1, 3])
axes[1, 3].set_title('결제 금액 (처리 후)')
axes[1, 3].set_xlabel('금액 (R$)')

# 전체 레이아웃 조정
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()



### 데이터 병합 merge 및 중복값 제거 

In [ ]:

# orders와 customers 병합 (customer_id 기준)
df_merged = pd.merge(orders, customers, on='customer_id', how='left')

# df_merged와 order_items 병합 (order_id 기준)
df_merged = pd.merge(df_merged, order_items, on='order_id', how='left')

# df_merged와 payments 병합 (order_id 기준)
df_merged = pd.merge(df_merged, payments, on='order_id', how='left')

# df_merged와 reviews 병합 (order_id 기준)
df_merged = pd.merge(df_merged, reviews, on='order_id', how='left')

# df_merged와 products 병합 (product_id 기준)
df_merged = pd.merge(df_merged, products, on='product_id', how='left')

# df_merged와 sellers 병합 (seller_id 기준)
df_merged = pd.merge(df_merged, sellers, on='seller_id', how='left')

# df_merged와 category_name 병합 (product_category_name 기준)
df_merged = pd.merge(df_merged, category_name, on='product_category_name', how='left')

# geolocation 데이터는 다대다 관계이므로, 추후에 필요에 따라 병합
# 현재는 geolocation_zip_code_prefix를 기준으로 고객 및 판매자 지역을 분석

# 최종 데이터프레임 미리보기
print("\n\n=== 모든 데이터셋 병합 후 최종 데이터프레임 ===")
print("데이터 크기:", df_merged.shape)
print(df_merged.head())

# 모든 컬럼이 동일한 중복 행을 제거합니다.
# 특히 여러 데이터셋을 병합하는 과정에서 중복이 발생할 수 있습니다.
initial_rows = df_merged.shape[0]
df_merged.drop_duplicates(inplace=True)
cleaned_rows = df_merged.shape[0]

print("\n=== 중복값 처리 현황 ===")
print(f"처리 전 행 수: {initial_rows}")
print(f"중복 제거 후 행 수: {cleaned_rows}")
print(f"제거된 중복 행 수: {initial_rows - cleaned_rows}")


### 추가 파생 변수 생성 

In [ ]:
df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])
df_merged['order_delivered_customer_date'] = pd.to_datetime(df_merged['order_delivered_customer_date'])
df_merged['order_estimated_delivery_date'] = pd.to_datetime(df_merged['order_estimated_delivery_date'])

#추가 파생 변수 생성 
# 배송 소요 시간 (delivery_time_days)
df_merged['delivery_time_days'] = (df_merged['order_delivered_customer_date'] - df_merged['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 배송 지연 여부 (delivery_delay)
df_merged['delivery_delay'] = (df_merged['order_delivered_customer_date'] - df_merged['order_estimated_delivery_date']).dt.total_seconds() / (24 * 3600)

# 구매 시간 관련 변수
df_merged['order_purchase_year'] = df_merged['order_purchase_timestamp'].dt.year
df_merged['order_purchase_month'] = df_merged['order_purchase_timestamp'].dt.month
df_merged['order_purchase_dayofweek'] = df_merged['order_purchase_timestamp'].dt.dayofweek
df_merged['order_purchase_hour'] = df_merged['order_purchase_timestamp'].dt.hour



df_merged.to_csv("olist_master_clean.csv", index=False)



***코드 진행 내용 요약***

제출해주신 코드는 크게 네 가지 단계로 진행되었습니다.

1. 데이터 불러오기 및 결측치 확인: 필요한 라이브러리를 임포트하고 9개의 CSV 파일을 로드했습니다. orders 데이터셋의 결측치를 확인하고, 배송 및 결제 관련 결측 행을 제거했습니다. reviews 데이터셋의 코멘트 결측치는 'no comment'로 채웠습니다.

2. 이상값 처리 및 시각화: orders (delivery_time_days), order_items (price), products (product_weight_g), payments (payment_value) 데이터셋의 주요 수치형 변수들에 대해 IQR(사분위 범위)을 활용한 이상값 처리를 진행했습니다. 이상값 처리 전후의 변화를 2행 4열의 박스플롯으로 시각화하여 한눈에 비교할 수 있도록 했습니다.

3. 데이터 병합: customers, orders, order_items, payments 데이터셋을 순차적으로 병합하여 하나의 통합된 데이터프레임(df_merged)을 만들었습니다. 이 과정에서 order_id에 결측치가 있는 행을 제거했습니다.

4. 파생 변수 생성 및 저장: 병합된 데이터프레임에서 delivery_days, delivery_delay, order_year, order_month, order_dayofweek와 같은 새로운 파생 변수들을 생성했습니다. 마지막으로 이 데이터를 olist_master_clean.csv 파일로 저장했습니다.

